In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_excel('/content/drive/MyDrive/Dataset.xlsx')
dataset.shape
#dataset.head()

(7653, 9)

In [3]:
X = dataset.drop('klass', axis=1)
Y = dataset['klass']

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

from random import randint

In [5]:
def Neural(layers, a, seed, alg):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size, random_state=seed)

  classifier = MLPClassifier(solver=alg, alpha=a, hidden_layer_sizes=layers, random_state=seed)
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)

  # scaler = StandardScaler() # Нормализация
  # scaler.fit(X_train)
  # X_train_norm = scaler.transform(X_train)
  # X_test_norm = scaler.transform(X_test)

  # classifier = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(6, 3), random_state=1)
  # classifier.fit(X_train_norm, y_train)
  # y_pred_norm = classifier.predict(X_test_norm)
  
  return y_test, y_pred, y_pred_norm

In [6]:
alg = 'adam'

alphas = []
norm = [] # 0 - без нормализации, 1 - с нормализацией 
tn = [] # истинно отрицательный
tp = [] # истинно положительный
fn = [] # ложно отрицательный
fp = [] # ложно положительный
fails = [] # ошибки за один проход
seed = [] # семя выборки
knot1 = [] # Количество узлов в 1-м слое
knot2 = []
knot3 = []
all = 7653
percent = [] # процент правильной выборки
test_size = 0.2  
test_size_num = 1531

In [7]:
def Test():
  y_test, y_pred, y_pred_norm = Neural(layers, alpha, random, alg)

  # norm.append(0)
  matrix = confusion_matrix(y_test, y_pred)
  tn.append(matrix[0][0])
  fn.append(matrix[0][1])
  fp.append(matrix[1][0])
  tp.append(matrix[1][1])
  fails.append(matrix[0][1] + matrix[1][0])
  percent.append(1 - (matrix[0][1] + matrix[1][0])/test_size_num)

  # norm.append(1)
  # matrix = confusion_matrix(y_test, y_pred_norm)
  # tn.append(matrix[0][0])
  # fn.append(matrix[0][1])
  # fp.append(matrix[1][0])
  # tp.append(matrix[1][1])
  # fails.append(matrix[0][1] + matrix[1][0])
  # percent.append(1 - (matrix[0][1] + matrix[1][0])/test_size_num)

  for i in range(0,1):  #2):
    seed.append(random)
    alphas.append(alpha)
    knot1.append(k1)
    if len(layers) > 1:
      knot2.append(k2)
    else:
      knot2.append(0)
    if len(layers) > 2:
      knot3.append(k3)
    else:
      knot3.append(0)
    

In [ ]:
for s in range(0, 20): # сколько раз попробовать каждый параметр
  random = randint(1, 2147483647)
  for alpha in [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]:
    for k1 in range(3, 11):
      for k2 in range(3, 11):
          layers = [k1, k2]
          Test()

In [ ]:
df = pd.DataFrame({'Knot1': knot1,
                   'Knot2': knot2,
                   'Alpha': alphas,
                   'TrueNegatives': tn,
                   'TruePositives': tp,
                   'FalseNegatives': fn,
                   'FalsePositives': fp,
                   'Fails': fails,
                   'PercentTrue': percent,
                  #  'IsNormalization': norm,
                   'Seed': seed})
df.to_excel('./Analysis.xlsx', index=False)

In [ ]:
sPer = []
stn = []
stp = []
sfn = []
sfp = []
sknot1 = []
sknot2 = []
for i in range(0, 8):
  for j in range(0, 8):
    sPer.append(0)
    stn.append(0)
    stp.append(0)
    sfn.append(0)
    sfp.append(0)
    sknot1.append(i + 3)
    sknot2.append(j + 3)
for i in range(0, len(percent)):
  sPer[knot1[i] - 3 + 8*(knot2[i] - 3)] += percent[i]
  stn[knot1[i] - 3 + 8*(knot2[i] - 3)] += tn[i]
  stp[knot1[i] - 3 + 8*(knot2[i] - 3)] += tp[i]
  sfn[knot1[i] - 3 + 8*(knot2[i] - 3)] += fn[i]
  sfp[knot1[i] - 3 + 8*(knot2[i] - 3)] += fp[i]
for i in range(0, len(sPer)):
  sPer[i] /=20*5
  stn[i] /=20*5
  stp[i] /=20*5
  sfn[i] /=20*5
  sfp[i] /=20*5
newdf = pd.DataFrame({'Knot1': sknot1,
                   'Knot2': sknot2,
                   'TrueNegatives': stn,
                   'TruePositives': stp,
                   'FalseNegatives': sfn,
                   'FalsePositives': sfp,
                   'PercentTrue': sPer})
newdf.to_excel('./Knots.xlsx', index=False)

In [ ]:
sPer = []
stn = []
stp = []
sfn = []
sfp = []
for i in range(0, 5):
  sPer.append(0)
  stn.append(0)
  stp.append(0)
  sfn.append(0)
  sfp.append(0)
for i in range(0, len(percent)):
  if alphas[i] == 1e-2:
    sPer[0] += percent[i]
    stn[0] += tn[i]
    stp[0] += tp[i]
    sfn[0] += fn[i]
    sfp[0] += fp[i]
  elif alphas[i] == 1e-3:
    sPer[1] += percent[i]
    stn[1] += tn[i]
    stp[1] += tp[i]
    sfn[1] += fn[i]
    sfp[1] += fp[i]
  elif alphas[i] == 1e-4:
    sPer[2] += percent[i]
    stn[2] += tn[i]
    stp[2] += tp[i]
    sfn[2] += fn[i]
    sfp[2] += fp[i]
  elif alphas[i] == 1e-5:
    sPer[3] += percent[i]
    stn[3] += tn[i]
    stp[3] += tp[i]
    sfn[3] += fn[i]
    sfp[3] += fp[i]
  else:
    sPer[4] += percent[i]
    stn[4] += tn[i]
    stp[4] += tp[i]
    sfn[4] += fn[i]
    sfp[4] += fp[i]

for i in range(0, len(sPer)):
  sPer[i] /=20*8*8
  stn[i] /=20*8*8
  stp[i] /=20*8*8
  sfn[i] /=20*8*8
  sfp[i] /=20*8*8
newdf = pd.DataFrame({'Alphas': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
                   'TrueNegatives': stn,
                   'TruePositives': stp,
                   'FalseNegatives': sfn,
                   'FalsePositives': sfp,
                   'PercentTrue': sPer})
newdf.to_excel('./Alphas.xlsx', index=False)